## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:



**Data From The General Social Survey: http://gss.norc.org/**
>Since 1972, the General Social Survey (GSS) has provided politicians, policymakers, and scholars with a clear and unbiased perspective on what Americans think and feel about such issues as national spendi​ng priorities, crime and punishment, intergroup relations, and confidence in institutions.
>> **The GSS gathers data on contemporary American society in order to monitor and explain trends and constants in attitudes, behaviors, and attributes.**  Hundreds of trends have been tracked since 1972. In addition, since the GSS adopted questions from earlier surveys, trends can be followed for up to 80 years.<br><br>
>> The GSS contains a **standard core of demographic, behavioral, and attitudinal questions, plus topics of special interest.** Among the topics covered are *civil liberties, crime and violence, intergroup tolerance, morality, national spending priorities, psychological well-being, social mobility, and stress and traumatic events.*<br><br>
>> **Altogether the GSS is the single best source for sociological and attitudinal trend data covering the United States.**
>> It allows researchers to examine the *structure and functioning of society in general* as well as the *role played by relevant subgroups* and to *compare the United States to other nations*.<br><br>
The GSS aims to make high-quality data easily accessible to scholars, students, policy makers, and others, with minimal cost and waiting.

- http://gss.norc.org/faq
- http://gss.norc.org/get-documentation

In [1]:
from bs_ds import *

,Module/Package Handle
pandas,pd
numpy,np
matplotlib,mpl
matplotlib.pyplot,plt
seaborn,sns
scip.stats.,sts


View our documentation at https://bs-ds.readthedocs.io/en/latest/bs_ds.html
Recommended import method:
 from bs_ds import *



In [3]:
# Your code here - remember to use markdown cells for comments as well!
df = pd.read_csv('dataset/reborn.csv')
inspect_df(df)

,Unnamed: 0,year,id_,degree,padeg,madeg,dipged,sex,partyid,vote12,pres12,if12who,polviews,natspac,natenvir,natheal,natcity,natcrime,natdrug,nateduc,natrace,natarms,nataid,natfare,natroad,spkath,colath,libath,spkrac,colrac,librac,spkcom,libcom,spkmil,colmil,libmil,spkhomo,colhomo,libhomo,spkmslm,colmslm,libmslm,gunlaw,grass,fund,fund16,spfund,racopen,coneduc,conjudge,consci,obey,popular,thnkself,workhard,helpoth,richwork,getahead,fepol,abany,sexeduc,premarsx,teensex,xmarsex,homosex,pornlaw,letdie1,suicide1,suicide2,suicide3,suicide4,polhitok,owngun,pistol,shotgun,rifle,rowngun,news,racdif2,racdif3,reborn,workwhts,workblks,workhsps,compuse,wkracism,wksexism,toofast,advfront,astrosci,scibnfts,balpos,balneg,scistudy,scitext,expdesgn,hotcore,radioact,boyorgrl,lasers,electron,viruses,bigbang,condrift,evolved,earthsun,solarrev,colsci,colscinm,intintl,intsci,intenvir,intmil,vissci,scinews3,scientgo,scienthe,scientod,scientbe,marhomo,xmarsex1,homosex1,concong,clergvte,trustsci,relext1,partners,matesex,frndsex,acqntsex,pikupsex,paidsex,othersex,sexsex,sexfreq,numwomen,nummen,partnrs5,sexsex5,evpaidsx,evstray,condom,relatsex,evidu,idu30,evcrack,crack30,hivtest,sexornt,genetst1,genegoo2,wordsum,intsex,ballot,hunt1,vote16,pres16,if16who,fammhneg,diagnosd,mhtreatd,abmoral,abhelp1,abhelp2,abhelp3,abinspay,knwcop,partpart,relext3,sexbirth,sexnow,geneabrt2
0,62466,2018,1,Junior college,High school,High school,High school diploma,Male,Not str republican,Voted,Romney,Not applicable,Conservative,Too much,About right,Too little,Too little,Too little,Too little,Too little,Too little,Too much,Too much,About right,Too little,Allowed,Allowed,Not remove,Allowed,Allowed,Not remove,Allowed,Not remove,Allowed,Allowed,Not remove,Allowed,Allowed,Not remove,"Yes, allowed","Yes, allowed",Not remove,Favor,Not applicable,Moderate,Fundamentalist,Not applicable,Owner decides,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Continue working,Hard work,Agree,No,Favor,Not wrong at all,Almst always wrg,Always wrong,Always wrong,Not applicable,No,Yes,Yes,Yes,Yes,Not applicable,No,No,No,No,Not applicable,Less than once wk,NO,No,Yes,4,6,2,Yes,No,No,Disagree,Strongly agree,Not at all scientific,Harmful results greater,Not applicable,Slightly in favor,Little understanding,Not applicable,Dont know,True,False,True,False,True,True,False,True,False,Earth around sun,One year,No,Not applicable,Moderately interested,Very interested,Very interested,Moderately interested,0,Science site,Strongly agree,Agree,Agree,Agree,Strongly disagree,Always wrong,Always wrong,Some confidence,Disagree,Strongly agree,Definitely not,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not very much,Good > harm,9,Female,Ballot a,Neither hunts,Voted,Trump,Not applicable,Not applicable,Not applicable,Not applicable,Morally opposed,Yes,Yes,Yes,People should be able,Not applicable,Not applicable,Probably not,Not applicable,Not applicable,No
1,62467,2018,2,High school,Lt high school,Lt high school,Ged,Female,"Ind,near dem",Voted,Obama,Not applicable,Don't know,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,About right,Allowed,Not allowed,Not remove,Allowed,Not allowed,Not remove,Allowed,Not remove,Not allowed,Not allowed,Not remove,Allowed,Allowed,Not remove,"Yes, allowed",Not allowed,Not remove,Favor,Not legal,Moderate,Moderate,Not applicable,Cant discriminate,A great deal,A great deal,Don't know,4th important,Least important,Most important,3rd important,2nd important,Not applicable,Hard work,Not applicable,Yes,Not applicable,Not ap

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2348 entries, 0 to 2347
Columns: 172 entries, Unnamed: 0 to geneabrt2
dtypes: int64(3), object(169)
memory usage: 3.1+ MB


None

,Unnamed: 0,year,id_
count,2348.000,2348.0,2348.000
mean,63639.500,2018.0,1174.500
std,677.954,0.0,677.954
min,62466.000,2018.0,1.000
25%,63052.750,2018.0,587.750
50%,63639.500,2018.0,1174.500
75%,64226.250,2018.0,1761.250
max,64813.000,2018.0,2348.000
